In [28]:
import sys
import pathlib
utils_path = pathlib.Path().absolute().parent.parent
sys.path.append(utils_path.__str__())

import utils.layout as lay
from utils.config import apikey
from eod import EodHistoricalData
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
pio.templates.default = 'simple_white+blog_mra'

In [3]:
def format_value(val):
    if not pd.isna(val) or val  != "nan":
        return f'{val / 1e6:.2f}'

In [4]:
client = EodHistoricalData(apikey)

In [5]:
stock = "TSLA.US"
peers = ["GM.US", "GOOGL.US", "AMZN.US", "TCEHY.US", "INTC.US", "AAPL.US", "F.US"]
all_ticks = [stock] + peers

df_fund = pd.DataFrame()
for tick in all_ticks:
    print(tick)
    inc_stat = client.get_fundamental_equity(tick, filter_='Financials::Income_Statement::yearly') # Stock
    df_temp = pd.DataFrame.from_dict(inc_stat, orient="index").sort_index()
    df_temp.index = pd.to_datetime(df_temp.index).year.astype(str)
    df_temp["ticker"] = tick
    df_temp = df_temp.set_index([df_temp["ticker"], df_temp.index])
    df_fund= pd.concat([df_fund, df_temp],axis = 0)

df_fund = df_fund.replace(to_replace={None: np.nan, type(None):  np.nan}, inplace=False)
df_fund.index.names = ["ticker", "year"]

TSLA.US
GM.US
GOOGL.US
AMZN.US
TCEHY.US
INTC.US
AAPL.US
F.US


In [6]:
df_fund = df_fund[["totalRevenue", "netIncome", "ebitda"]].astype(float)
df_fund["revGrowth"] = df_fund.groupby("ticker")["totalRevenue"].pct_change()
df_fund["ebitdaMargin"] = df_fund["ebitda"] / df_fund["totalRevenue"]
df_fund["profitMargin"] = df_fund["netIncome"] / df_fund["totalRevenue"]
df_fund = df_fund[df_fund.index.get_level_values(1) != "2023"] ## adjust for fiscal year
df_last = df_fund.groupby(level=['ticker']).tail(3)
df_last

totalRevenue     netIncome        ebitda  revGrowth  \
ticker   year                                                        
TSLA.US  2020  3.153600e+10  6.900000e+08  4.224000e+09   0.283099   
         2021  5.382300e+10  5.519000e+09  9.598000e+09   0.706716   
         2022  8.146200e+10  1.258300e+10  1.783300e+10   0.513517   
GM.US    2020  1.224850e+11  6.427000e+09  2.174300e+10  -0.107493   
         2021  1.270040e+11  1.001900e+10  2.571700e+10   0.036894   
         2022  1.567350e+11  9.934000e+09  2.387400e+10   0.234095   
GOOGL.US 2020  1.825270e+11  4.026900e+10  5.492100e+10   0.127705   
         2021  2.576370e+11  7.603300e+10  9.115500e+10   0.411501   
         2022  2.828360e+11  5.997200e+10  9.077000e+10   0.097808   
AMZN.US  2020  3.860640e+11  2.133100e+10  4.807900e+10   0.376234   
         2021  4.698220e+11  3.336400e+10  5.931200e+10   0.216954   
         2022  5.139830e+11 -2.722000e+09  5.416900e+10   0.093995   
TCEHY.US 2020  4.820640e+11  1.598470e+11  1.841880e+11   0.277705   
         2021  5.601180e+11  2.248220e+11  1.662470e+11   0.161916   
         2022  5.545520e+11  1.882430e+11  1.491310e+11  -0.009937   
INTC.US  2020  7.786700e+10  2.089900e+10  3.801900e+10   0.082012   
         2021  7.902400e+10  1.986800e+10  3.650000e+10   0.014859   
         2022  6.305400e+10  8.014000e+09  1.110500e+10  -0.202091   
AAPL.US  2020  2.745150e+11  5.741100e+10  7.734400e+10   0.055121   
         2021  3.658170e+11  9.468000e+10  1.202330e+11   0.332594   
         2022  3.943280e+11  9.980300e+10  1.305410e+11   0.077938   
F.US     2020  1.271440e+11 -1.276000e+09  1.627500e+10  -0.184452   
         2021  1.363410e+11  1.793700e+10  2.333800e+10   0.072335   
         2022  1.580570e+11 -2.152000e+09  1.889400e+10   0.159277   

               ebitdaMargin  profitMargin  
ticker   year                              
TSLA.US  2020      0.133942      0.021880  
         2021      0.178325      0.102540  
         2022      0.218912      0.154465  
GM.US    2020      0.177516      0.052472  
         2021      0.202490      0.078887  
         2022      0.152321      0.063381  
GOOGL.US 2020      0.300892      0.220619  
         2021      0.353812      0.295117  
         2022      0.320928      0.212038  
AMZN.US  2020      0.124536      0.055252  
         2021      0.126244      0.071014  
         2022      0.105391     -0.005296  
TCEHY.US 2020      0.382082      0.331589  
         2021      0.296807      0.401383  
         2022      0.268922      0.339451  
INTC.US  2020      0.488256      0.268394  
         2021      0.461885      0.251417  
         2022      0.176119      0.127097  
AAPL.US  2020      0.281748      0.209136  
         2021      0.328670      0.258818  
         2022      0.331047      0.253096  
F.US     2020      0.128004     -0.010036  
         2021      0.171174      0.131560  
         2022      0.119539     -0.013615

In [7]:
df_data = pd.DataFrame(index =all_ticks)
marketCap = np.zeros(len(all_ticks))
enterpriseValue = np.zeros(len(all_ticks))

c = 0
for tick in all_ticks:
    print(tick)
    marketCap[c] =  client.get_fundamental_equity(tick, filter_='Highlights::MarketCapitalization')
    enterpriseValue[c] = client.get_fundamental_equity(tick, filter_='Valuation::EnterpriseValue')
    c+=1
    
df_data["marketCap"] = marketCap
df_data["enterpriseValue"] = enterpriseValue
df_data

TSLA.US
GM.US
GOOGL.US
AMZN.US
TCEHY.US
INTC.US
AAPL.US
F.US


,marketCap,enterpriseValue
TSLA.US,7.488900e+11,7.320293e+11
GM.US,4.528870e+10,1.407687e+11
GOOGL.US,1.626126e+12,1.528454e+12
AMZN.US,1.496777e+12,1.598677e+12
TCEHY.US,3.767392e+11,4.574669e+11
INTC.US,1.784199e+11,2.023966e+11
AAPL.US,3.008222e+12,2.995691e+12
F.US,4.283156e+10,1.565066e+11


In [8]:
df_all = pd.merge(df_data, df_last.reset_index(level=1, drop=False), left_index=True, right_index=True)
df_all = df_all.set_index([df_all.index, "year"])
df_all.index.names = ["ticker", "year"]
df_all

marketCap  enterpriseValue  totalRevenue     netIncome  \
ticker   year                                                              
AAPL.US  2020  3.008222e+12     2.995691e+12  2.745150e+11  5.741100e+10   
         2021  3.008222e+12     2.995691e+12  3.658170e+11  9.468000e+10   
         2022  3.008222e+12     2.995691e+12  3.943280e+11  9.980300e+10   
AMZN.US  2020  1.496777e+12     1.598677e+12  3.860640e+11  2.133100e+10   
         2021  1.496777e+12     1.598677e+12  4.698220e+11  3.336400e+10   
         2022  1.496777e+12     1.598677e+12  5.139830e+11 -2.722000e+09   
F.US     2020  4.283156e+10     1.565066e+11  1.271440e+11 -1.276000e+09   
         2021  4.283156e+10     1.565066e+11  1.363410e+11  1.793700e+10   
         2022  4.283156e+10     1.565066e+11  1.580570e+11 -2.152000e+09   
GM.US    2020  4.528870e+10     1.407687e+11  1.224850e+11  6.427000e+09   
         2021  4.528870e+10     1.407687e+11  1.270040e+11  1.001900e+10   
         2022  4.528870e+10     1.407687e+11  1.567350e+11  9.934000e+09   
GOOGL.US 2020  1.626126e+12     1.528454e+12  1.825270e+11  4.026900e+10   
         2021  1.626126e+12     1.528454e+12  2.576370e+11  7.603300e+10   
         2022  1.626126e+12     1.528454e+12  2.828360e+11  5.997200e+10   
INTC.US  2020  1.784199e+11     2.023966e+11  7.786700e+10  2.089900e+10   
         2021  1.784199e+11     2.023966e+11  7.902400e+10  1.986800e+10   
         2022  1.784199e+11     2.023966e+11  6.305400e+10  8.014000e+09   
TCEHY.US 2020  3.767392e+11     4.574669e+11  4.820640e+11  1.598470e+11   
         2021  3.767392e+11     4.574669e+11  5.601180e+11  2.248220e+11   
         2022  3.767392e+11     4.574669e+11  5.545520e+11  1.882430e+11   
TSLA.US  2020  7.488900e+11     7.320293e+11  3.153600e+10  6.900000e+08   
         2021  7.488900e+11     7.320293e+11  5.382300e+10  5.519000e+09   
         2022  7.488900e+11     7.320293e+11  8.146200e+10  1.258300e+10   

                     ebitda  revGrowth  ebitdaMargin  profitMargin  
ticker   year                                                       
AAPL.US  2020  7.734400e+10   0.055121      0.281748      0.209136  
         2021  1.202330e+11   0.332594      0.328670      0.258818  
         2022  1.305410e+11   0.077938      0.331047      0.253096  
AMZN.US  2020  4.807900e+10   0.376234      0.124536      0.055252  
         2021  5.931200e+10   0.216954      0.126244      0.071014  
         2022  5.416900e+10   0.093995      0.105391     -0.005296  
F.US     2020  1.627500e+10  -0.184452      0.128004     -0.010036  
         2021  2.333800e+10   0.072335      0.171174      0.131560  
         2022  1.889400e+10   0.159277      0.119539     -0.013615  
GM.US    2020  2.174300e+10  -0.107493      0.177516      0.052472  
         2021  2.571700e+10   0.036894      0.202490      0.078887  
         2022  2.387400e+10   0.234095      0.152321      0.063381  
GOOGL.US 2020  5.492100e+10   0.127705      0.300892      0.220619  
         2021  9.115500e+10   0.411501      0.353812      0.295117  
         2022  9.077000e+10   0.097808      0.320928      0.212038  
INTC.US  2020  3.801900e+10   0.082012      0.488256      0.268394  
         2021  3.650000e+10   0.014859      0.461885      0.251417  
         2022  1.110500e+10  -0.202091      0.176119      0.127097  
TCEHY.US 2020  1.841880e+11   0.277705      0.382082      0.331589  
         2021  1.662470e+11   0.161916      0.296807      0.401383  
         2022  1.491310e+11  -0.009937      0.268922      0.339451  
TSLA.US  2020  4.224000e+09   0.283099      0.133942      0.021880  
         2021  9.598000e+09   0.706716      0.178325      0.102540  
         2022  1.783300e+10   0.513517      0.218912      0.154465

In [9]:
### Trading Multiple
df_all["EVofRevenue"] = df_all["enterpriseValue"] / df_all["totalRevenue"]
df_all["EVofEbitda"] = df_all["enterpriseValue"] / df_all["ebitda"]
df_all["PofE"] = df_all["marketCap"] / df_all["netIncome"]
df_all

marketCap  enterpriseValue  totalRevenue     netIncome  \
ticker   year                                                              
AAPL.US  2020  3.008222e+12     2.995691e+12  2.745150e+11  5.741100e+10   
         2021  3.008222e+12     2.995691e+12  3.658170e+11  9.468000e+10   
         2022  3.008222e+12     2.995691e+12  3.943280e+11  9.980300e+10   
AMZN.US  2020  1.496777e+12     1.598677e+12  3.860640e+11  2.133100e+10   
         2021  1.496777e+12     1.598677e+12  4.698220e+11  3.336400e+10   
         2022  1.496777e+12     1.598677e+12  5.139830e+11 -2.722000e+09   
F.US     2020  4.283156e+10     1.565066e+11  1.271440e+11 -1.276000e+09   
         2021  4.283156e+10     1.565066e+11  1.363410e+11  1.793700e+10   
         2022  4.283156e+10     1.565066e+11  1.580570e+11 -2.152000e+09   
GM.US    2020  4.528870e+10     1.407687e+11  1.224850e+11  6.427000e+09   
         2021  4.528870e+10     1.407687e+11  1.270040e+11  1.001900e+10   
         2022  4.528870e+10     1.407687e+11  1.567350e+11  9.934000e+09   
GOOGL.US 2020  1.626126e+12     1.528454e+12  1.825270e+11  4.026900e+10   
         2021  1.626126e+12     1.528454e+12  2.576370e+11  7.603300e+10   
         2022  1.626126e+12     1.528454e+12  2.828360e+11  5.997200e+10   
INTC.US  2020  1.784199e+11     2.023966e+11  7.786700e+10  2.089900e+10   
         2021  1.784199e+11     2.023966e+11  7.902400e+10  1.986800e+10   
         2022  1.784199e+11     2.023966e+11  6.305400e+10  8.014000e+09   
TCEHY.US 2020  3.767392e+11     4.574669e+11  4.820640e+11  1.598470e+11   
         2021  3.767392e+11     4.574669e+11  5.601180e+11  2.248220e+11   
         2022  3.767392e+11     4.574669e+11  5.545520e+11  1.882430e+11   
TSLA.US  2020  7.488900e+11     7.320293e+11  3.153600e+10  6.900000e+08   
         2021  7.488900e+11     7.320293e+11  5.382300e+10  5.519000e+09   
         2022  7.488900e+11     7.320293e+11  8.146200e+10  1.258300e+10   

                     ebitda  revGrowth  ebitdaMargin  profitMargin  \
ticker   year                                                        
AAPL.US  2020  7.734400e+10   0.055121      0.281748      0.209136   
         2021  1.202330e+11   0.332594      0.328670      0.258818   
         2022  1.305410e+11   0.077938      0.331047      0.253096   
AMZN.US  2020  4.807900e+10   0.376234      0.124536      0.055252   
         2021  5.931200e+10   0.216954      0.126244      0.071014   
         2022  5.416900e+10   0.093995      0.105391     -0.005296   
F.US     2020  1.627500e+10  -0.184452      0.128004     -0.010036   
         2021  2.333800e+10   0.072335      0.171174      0.131560   
         2022  1.889400e+10   0.159277      0.119539     -0.013615   
GM.US    2020  2.174300e+10  -0.107493      0.177516      0.052472   
         2021  2.571700e+10   0.036894      0.202490      0.078887   
         2022  2.387400e+10   0.234095      0.152321      0.063381   
GOOGL.US 2020  5.492100e+10   0.127705      0.300892      0.220619   
         2021  9.115500e+10   0.411501      0.353812      0.295117   
         2022  9.077000e+10   0.097808      0.320928      0.212038   
INTC.US  2020  3.801900e+10   0.082012      0.488256      0.268394   
         2021  3.650000e+10   0.014859      0.461885      0.251417   
         2022  1.110500e+10  -0.202091      0.176119      0.127097   
TCEHY.US 2020  1.841880e+11   0.277705      0.382082      0.331589   
         2021  1.662470e+11   0.161916      0.296807      0.401383   
         2022  1.491310e+11  -0.009937      0.268922      0.339451   
TSLA.US  2020  4.224000e+09   0.283099      0.133942      0.021880   
         2021  9.598000e+09   0.706716      0.178325      0.102540   
         2022  1.783300e+10   0.513517      0.218912      0.154465   

               EVofRevenue  EVofEbitda         PofE  
ticker   year                                        
AAPL.US  2020    10.912667   38.732039    52.398015  
         2021     8.189042   24.915712    31.772523  
    

In [10]:
df_comp = df_all[["totalRevenue", "ebitda", "netIncome", "revGrowth", "ebitdaMargin", "profitMargin", "EVofRevenue", "EVofEbitda", "PofE"]].unstack(level='year')
df_comp.loc["Minimum", :] = df_comp.min()
df_comp.loc["Mean", :] = df_comp.mean()
df_comp.loc[" Median", :] = df_comp.median()
df_comp.loc["Maximum", :] = df_comp.max()
df_comp.loc[:, "revGrowth":]

revGrowth                     ebitdaMargin                      \
year          2020      2021      2022         2020      2021      2022   
ticker                                                                    
AAPL.US   0.055121  0.332594  0.077938     0.281748  0.328670  0.331047   
AMZN.US   0.376234  0.216954  0.093995     0.124536  0.126244  0.105391   
F.US     -0.184452  0.072335  0.159277     0.128004  0.171174  0.119539   
GM.US    -0.107493  0.036894  0.234095     0.177516  0.202490  0.152321   
GOOGL.US  0.127705  0.411501  0.097808     0.300892  0.353812  0.320928   
INTC.US   0.082012  0.014859 -0.202091     0.488256  0.461885  0.176119   
TCEHY.US  0.277705  0.161916 -0.009937     0.382082  0.296807  0.268922   
TSLA.US   0.283099  0.706716  0.513517     0.133942  0.178325  0.218912   
Minimum  -0.184452  0.014859 -0.202091     0.124536  0.126244  0.105391   
Mean      0.080609  0.218736  0.084724     0.237946  0.249517  0.199841   
 Median   0.081310  0.189435  0.089359     0.207731  0.226003  0.187980   
Maximum   0.376234  0.706716  0.513517     0.488256  0.461885  0.331047   

         profitMargin                     EVofRevenue                       \
year             2020      2021      2022        2020       2021      2022   
ticker                                                                       
AAPL.US      0.209136  0.258818  0.253096   10.912667   8.189042  7.596952   
AMZN.US      0.055252  0.071014 -0.005296    4.140963   3.402729  3.110369   
F.US        -0.010036  0.131560 -0.013615    1.230940   1.147906  0.990191   
GM.US        0.052472  0.078887  0.063381    1.149273   1.108380  0.898132   
GOOGL.US     0.220619  0.295117  0.212038    8.373853   5.932589  5.404030   
INTC.US      0.268394  0.251417  0.127097    2.599260   2.561204  3.209893   
TCEHY.US     0.331589  0.401383  0.339451    0.948975   0.816733  0.824931   
TSLA.US      0.021880  0.102540  0.154465   23.212498  13.600679  8.986145   
Minimum     -0.010036  0.071014 -0.013615    0.948975   0.816733  0.824931   
Mean         0.126586  0.184639  0.124111    5.946378   4.175111  3.538397   
 Median      0.090919  0.158099  0.125604    3.370112   2.981967  3.160131   
Maximum      0.331589  0.401383  0.339451   23.212498  13.600679  8.986145   

          EVofEbitda                               PofE              \
year            2020       2021       2022         2020        2021   
ticker                                                                
AAPL.US    38.732039  24.915712  22.948275    52.398015   31.772523   
AMZN.US    33.251044  26.953684  29.512764    70.169077   44.862024   
F.US        9.616382   6.706085   8.283403   -33.567057    2.387889   
GM.US       6.474210   5.473762   5.896320     7.046632    4.520282   
GOOGL.US   27.830053  16.767641  16.838761    40.381588   21.387110   
INTC.US     5.323565   5.545112  18.225718     8.537245    8.980264   
TCEHY.US    2.483695   2.751730   3.067550     2.356874    1.675722   
TSLA.US   173.302396  76.268944  41.049140  1085.347775  135.693054   
Minimum     2.483695   2.751730   3.067550   -33.567057    1.675722   
Mean       33.277453  18.681600  16.543276   133.233677   28.106066   
 Median    18.723217  11.736863  16.691018    24.459417   15.183687   
Maximum   173.302396  76.268944  41.049140  1085.347775  135.693054   

                      
year            2022  
ticker                
AAPL.US    30.141604  
AMZN.US  -549.881180  
F.US      -19.903143  
GM.US       4.558959  
GOOGL.US   27.114756  
INTC.US    22.263525  
TCEHY.US    2.001345  
TSLA.US    59.516011  
Minimum  -549.881180  
Mean     -108.229923  
 Median     3.280152  
Maximum    59.516011

In [11]:
year = "2022"
df_comp.columns = df_comp.columns.map('_'.join)
df_comp = df_comp.loc[:, df_comp.columns.str.contains(year)]
df_comp.columns = [i[:-5] for i in df_comp.columns]
df_comp

,totalRevenue,ebitda,netIncome,revGrowth,ebitdaMargin,profitMargin,EVofRevenue,EVofEbitda,PofE
ticker,,,,,,,,,
AAPL.US,3.943280e+11,1.305410e+11,9.980300e+10,0.077938,0.331047,0.253096,7.596952,22.948275,30.141604
AMZN.US,5.139830e+11,5.416900e+10,-2.722000e+09,0.093995,0.105391,-0.005296,3.110369,29.512764,-549.881180
F.US,1.580570e+11,1.889400e+10,-2.152000e+09,0.159277,0.119539,-0.013615,0.990191,8.283403,-19.903143
GM.US,1.567350e+11,2.387400e+10,9.934000e+09,0.234095,0.152321,0.063381,0.898132,5.896320,4.558959
GOOGL.US,2.828360e+11,9.077000e+10,5.997200e+10,0.097808,0.320928,0.212038,5.404030,16.838761,27.114756
INTC.US,6.305400e+10,1.110500e+10,8.014000e+09,-0.202091,0.176119,0.127097,3.209893,18.225718,22.263525
TCEHY.US,5.545520e+11,1.491310e+11,1.882430e+11,-0.009937,0.268922,0.339451,0.824931,3.067550,2.001345
TSLA.US,8.146200e+10,1.783300e+10,1.258300e+10,0.513517,0.218912,0.154465,8.986145,41.049140,59.516011
Minimum,6.305400e+10,1.110500e+10,-2.722000e+09,-0.202091,0.105391,-0.013615,0.824931,3.067550,-549.881180


In [40]:
plot

,EVofRevenue,EVofEbitda,PofE
ticker,,,
AAPL.US,7.596952,22.948275,30.141604
AMZN.US,3.110369,29.512764,NaN
F.US,0.990191,8.283403,NaN
GM.US,0.898132,5.896320,4.558959
GOOGL.US,5.404030,16.838761,27.114756
INTC.US,3.209893,18.225718,22.263525
TCEHY.US,0.824931,3.067550,2.001345
TSLA.US,8.986145,41.049140,59.516011


In [44]:
plot = df_comp.loc[:"TSLA.US", "EVofRevenue":]
plot[plot < 0] = np.nan

fig = make_subplots(rows=1, cols=3,
                    subplot_titles=("EVofRevenue/EVofEbitda", "EVofRevenue/PofE", "EVofEbitda/PofE"))

fig.add_trace(
    go.Scatter(x=plot.EVofRevenue, y=plot.EVofEbitda, mode="markers+text",  
               text=list(plot.index), textposition="top center", showlegend=False), row=1, col=1)
fig.add_trace(
    go.Scatter(x=plot.EVofRevenue, y=plot.PofE, mode="markers+text", text=list(plot.index), textposition="top center",  
               showlegend=False), row=1, col=2,)
fig.add_trace(
    go.Scatter(x=plot.EVofEbitda, y=plot.PofE, mode="markers+text", text=list(plot.index), textposition="top center", 
               showlegend=False),   row=1, col=3,)
#fig.add_trace(go.Scatter(x=linspace, y=regline_msft,  showlegend=False), row=1, col=1)


fig.update_layout(height=600, width=1200, title_text="Trading Multiples")
fig.show()

In [12]:
outshares = client.get_fundamental_equity(stock, filter_='SharesStats::SharesOutstanding')
df_price = df_comp.loc["Minimum":, "EVofRevenue":]
# Financial metric
df_price[stock[:-3]+"_revenue"] = df_comp.loc[stock, "totalRevenue"]
df_price[stock[:-3]+"_ebitda"] = df_comp.loc[stock, "ebitda"]
df_price["EPS"] = df_comp.loc[stock, "netIncome"] / outshares
# Enterprise Value
df_price["EV_revenue"] = df_price[stock[:-3]+"_revenue"] *df_price["EVofRevenue"]
df_price["EV_ebitda"] = df_price[stock[:-3]+"_ebitda"] *df_price["EVofEbitda"]

bs = client.get_fundamental_equity(stock, filter_='Financials::Balance_Sheet::yearly')
df_price["cash"] = float(pd.DataFrame(bs).loc["cash"].values[0])
df_price["debt"] = float(pd.DataFrame(bs).loc["totalLiab"].values[0])
df_price["outshares"] = outshares

In [13]:
df_price.T

ticker,Minimum,Mean,Median,Maximum
EVofRevenue,8.249305e-01,3.538397e+00,3.160131e+00,8.986145e+00
EVofEbitda,3.067550e+00,1.654328e+01,1.669102e+01,4.104914e+01
PofE,-5.498812e+02,-1.082299e+02,3.280152e+00,5.951601e+01
TSLA_revenue,8.146200e+10,8.146200e+10,8.146200e+10,8.146200e+10
TSLA_ebitda,1.783300e+10,1.783300e+10,1.783300e+10,1.783300e+10
EPS,3.958263e+00,3.958263e+00,3.958263e+00,3.958263e+00
EV_revenue,6.720049e+10,2.882449e+11,2.574306e+11,7.320293e+11
EV_ebitda,5.470363e+10,2.950162e+11,2.976509e+11,7.320293e+11
cash,1.625300e+10,1.625300e+10,1.625300e+10,1.625300e+10
debt,3.644000e+10,3.644000e+10,3.644000e+10,3.644000e+10


In [14]:
df_price["EQ_revenue"] = df_price["EV_revenue"] +  df_price["cash"] - df_price["debt"]
df_price["EQ_ebitda"] = df_price["EV_ebitda"] +  df_price["cash"] - df_price["debt"]
df_price["EQ_netIncome"] = df_price["PofE"] *  df_price["EPS"] * df_price["outshares"]

df_price["ISP_revenue"] = df_price["EQ_revenue"] / df_price["outshares"]
df_price["ISP_ebitda"] = df_price["EQ_ebitda"] / df_price["outshares"]
df_price["ISP_netIncome"] = df_price["EQ_netIncome"] / df_price["outshares"]

In [49]:
df_out = df_price.T.loc["ISP_revenue":, :]
df_out = df_out.round(2)
df_out[df_out<0] = ""
df_out

ticker,Minimum,Mean,Median,Maximum
ISP_revenue,14.79,84.32,74.63,223.93
ISP_ebitda,10.86,86.45,87.28,223.93
ISP_netIncome,,,12.98,235.58
